# LIT-PCBA 
this dataset is a proposed "more difficult" test than DUD-E. I used a 75/25 (sklearn default) stratified split. The dataset is heavily imbalanced, like DUD-E.

In [9]:
import pickle
from sklearn.metrics import roc_auc_score
import numpy as np
import seaborn as sns
import torch
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from sklearn.metrics import f1_score, recall_score
import matplotlib.pyplot as plt
import sys
# sys.path.append('..')
sys.path.insert(0, "/g/g13/jones289/workspace/hd-cuda-master")
# print(sys.path)
import hdpy
import hdpy.ecfp
# from hdpy.analysis import load_pkl
from hdpy.metrics import compute_enrichment_factor
from pathlib import Path


plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
plt.rc('axes', labelsize=17)
plt.rc('figure', titlesize=20)


# SEED=125
SEED=5
# SEED=2

# Todo: visualize the learning curves 

In [10]:
data_p = Path(f"/p/vast1/jones289/hd_results/{SEED}")

In [11]:
green_color_pal = sns.color_palette("Greens", 10)
blue_color_pal = sns.color_palette("Blues", 10)
rocket_color_pal = sns.color_palette("rocket", 10)

In [12]:
model_dict = {
    "hdbind-rp-molformer": ("HDBind+MolFormer", rocket_color_pal[0]),
    "hdbind-rp-ecfp-1024-1": ("HDBind+ECFP-1024-1", rocket_color_pal[2]),
#     "hdbind-rp-ecfp-1024-2": ("HDBind+ECFP-1024-1", rocket_color_pal[2]),
#     "hdbind-rp-ecfp-1024-4": ("HDBind+ECFP-1024-1", rocket_color_pal[2]),
#     "hdbind-rp-ecfp-2048-1": ("HDBind+ECFP-1024-1", rocket_color_pal[2]),
#     "hdbind-rp-ecfp-2048-2": ("HDBind+ECFP-1024-1", rocket_color_pal[2]),
#     "hdbind-rp-ecfp-2048-4": ("HDBind+ECFP-1024-1", rocket_color_pal[2]),
    "molehd-bpe": ("MoleHD-BPE", rocket_color_pal[4]),
#     "smiles-pe.atomwise.0": rocket_color_pal[2],
#     "smiles-pe.bpe.0": rocket_color_pal[4],   
# "selfies.atomwise": green_color_pal[4],
#     "ecfp": green_color_pal[6],
#     "rp": green_color_pal[8],
#     "rf": blue_color_pal[4],
#     "mlp": blue_color_pal[7],
#     "HDC-MLP": green_color_pal[9],
#     "HDC-RF": green_color_pal[9],
#     "Vina": "salmon",
}


# model_order_list = [
#     ("hdbind-rp-molformer", )
#     ("smiles-pe.atomwise.0", "MoleHD-Atomw."),
#     ("smiles-pe.bpe.0", "MoleHD-BPE"),
#     ("smiles-pe.ngram.1", "SMILES uni-gram"),
#     ("selfies.atomwise", "HDBind-SELFIES"),
#     ("selfies.selfies-charwise", "SELFIES uni-gram"),
#     ("ecfp", "HDBind-ECFP"),
#     ("rp", "HDBind-ECFP+RP"),
#     ("rf", "RF"),
#     ("mlp", "MLP"),
#     ("Vina", "Vina")
# ]


# model_name_dict = {
#     "hdbind-rp-molformer": "HDBind+MolFormer"
#     "smiles-pe.atomwise.0": "MoleHD-Atomw.", 
#     "smiles-pe.bpe.0": "MoleHD-BPE",
#     "smiles-pe.ngram.1": "SMILES uni-gram",
#     "selfies.atomwise": "HDBind-SELFIES",
#     "selfies.selfies-charwise": "SELFIES uni-gram",
#     "ecfp": "HDBind-ECFP",
#     "rp": "HDBind-RPFP",
#     "rf": "RF",
#     "mlp": "MLP",
#     "Vina": "Vina",
#     "HDC-MLP": "HDC-MLP",
#     "HDC-RF": "HDC-RF"
# }

# LIT-PCBA Results

export CONDA_ROOT=/usr/workspace/jones289/anaconda3-power #installed using the LC installer script
export PATH=$CONDA_ROOT/bin:$PATH
export CONDA_EXE=$CONDA_ROOT/bin/conda
ml load gcc/11
export PYTHONPATH=$PWD:$PYTHONPATH
source activate /usr/workspace/jones289/anaconda3-power/envs/opence-1.8.0

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score


def compute_metrics(y_pred, y_score, y_true, p):
    
    return {"precision": precision_score(y_pred=y_pred, y_true=y_true, zero_division=0),
            "recall": recall_score(y_pred=y_pred, y_true=y_true),
           "f1": f1_score(y_pred=y_pred, y_true=y_true, zero_division=0),
            "enrich": float(compute_enrichment_factor(scores=y_score, labels=y_true, n_percent=p)),            
           "roc": roc_auc_score(y_score=y_score, y_true=y_true)
           }


def aggregate_results(dataset, range_limit=10, multistep_initial_p=None, 
                      multistep_p_list=None, 
                      multistep_sklearn_model=None):
    

    model_metric_dict = {"model": [], "enrich": [], "p":[], "train_time":[], "test_time": [], "target": [],
                        "seed": [], "recall": [], "precision": [], "f1": [], 
                        }

    for model_name, model_tup in tqdm(model_dict.items(), total=len(model_dict), position=0):
                        

        data_path_list = list(data_p.glob(f"{model_name}.{dataset}*.pkl"))
        print(f"{model_name}-{dataset}: {len(model_tup)}\t {len(data_path_list)}")

        

        for path in tqdm(data_path_list, total=len(data_path_list), position=1):

            
            with open(path, "rb") as handle:
                model_data_dict = pickle.load(handle)
            
            target = path.name.split(".")[1]
            
            print(target, path, model_name)


#             import ipdb
#             ipdb.set_trace()
            for seed in range(len(model_data_dict['trials'])):
                trial_dict = model_data_dict['trials'][seed]
            
                if "x_train" in trial_dict.keys():
                    del model_data_dict['trials'][seed]['x_train']
                
                if "x_test" in trial_dict.keys():
                    del model_data_dict['trials'][seed]['x_test']
                    
                with open(path, "wb") as handle:
                    pickle.dump(model_data_dict, handle)
                
                with open(path, "rb") as handle:
                    model_data_dict = pickle.load(handle)
                    
                    
                y_test = model_data_dict["y_test"]
#                 x_test = model_data_dict["x_test"]
                actives_database = sum(y_test)
                database_size = y_test.shape[0]
                
#                 eta = None 
                

#                 if model in ["rf", "mlp"]:


#                     for p in [.01, .1]:

#                         sklearn_model = model_data_dict[seed]["model"]

#                         y_score = sklearn_model.predict_proba(x_test)[:, 1]            
#                         y_pred = sklearn_model.predict(x_test)
#                         enrich = compute_enrichment_factor(sample_scores=y_score, 
#                                                 sample_labels=y_test,
#                                                 n_percent=p, 
#                                                 actives_database=actives_database, 
#                                                 database_size=database_size)

                
#                         metrics = compute_metrics(y_pred=y_pred, y_score=y_score, y_true=y_test)
                
#                         model_metric_dict["model"].append(model)
#                         model_metric_dict["target"].append(target)
#                         model_metric_dict["enrich"].append(enrich)
#                         model_metric_dict["p"].append(p)
#                         model_metric_dict["seed"].append(seed)
#                         model_metric_dict["precision"].append(metrics["precision"])
#                         model_metric_dict["recall"].append(metrics["recall"])
#                         model_metric_dict["f1"].append(metrics["f1"])
# #                         model_metric_dict["roc"].append(metrics["roc"])

                        
#                 elif model.lower() in ["hdc-rf", "hdc-mlp"]:

#                     sklearn_result_file = Path(f"{data_p}/{dataset.replace('-', '_')}.{target}.{multistep_sklearn_model}.None.{ngram_order}.pkl")

#                     with open(sklearn_result_file, "rb") as handle:
#                         sklearn_result_dict = pickle.load(handle)

#                     target_test_hv_path = f"{hd_cache_dir}/{target}/test_dataset_hv.pth"

#                     hv_test = torch.load(target_test_hv_path, map_location="cpu")
#                     hdc_conf_scores = model_data_dict[seed]["model"].compute_confidence(hv_test)

# #                     # filter the data
#                     idxs = np.flip(np.argsort(hdc_conf_scores.squeeze().cpu().numpy(), kind="stable"))

                    
#                     sample_n = int(multistep_initial_p * y_test.shape[0])
#                     samp_idxs = idxs[:sample_n]
                    

#                     x_test_samp = x_test[samp_idxs]
#                     y_test_samp = y_test[samp_idxs]
                                            
    
#                     for p in multistep_p_list:

#                         sklearn_model = sklearn_result_dict[seed]["model"]
            
#                         sklearn_scores_samp = sklearn_model.predict_proba(x_test_samp)[:, 1]                        
            
#                         y_pred = sklearn_model.predict(x_test_samp)
#                         enrich = compute_enrichment_factor(sample_scores=sklearn_scores_samp, 
#                                                 sample_labels=y_test_samp,
#                                                 n_percent=p, 
#                                                 actives_database=actives_database, 
#                                                 database_size=database_size)


#                         metrics = compute_metrics(y_pred=y_pred, y_score=sklearn_scores_samp, y_true=y_test_samp)
                
#                         model_metric_dict["model"].append(model)
#                         model_metric_dict["target"].append(target)
#                         model_metric_dict["enrich"].append(enrich)
#                         model_metric_dict["p"].append(round(multistep_initial_p * p, 2))
#                         model_metric_dict["seed"].append(seed)
#                         model_metric_dict["precision"].append(metrics["precision"])
#                         model_metric_dict["recall"].append(metrics["recall"])
#                         model_metric_dict["f1"].append(metrics["f1"])
# #                         model_metric_dict["roc"].append(metrics["roc"])
                    

#                 else:                  
                    
#                 target_test_hv_path = f"{hd_cache_dir}/{target}/test_dataset_hv.pth"
                
#                 hv_test = torch.load(target_test_hv_path, map_location="cpu")
#                 hdc_conf_scores = model_data_dict[seed]["model"].compute_confidence(hv_test)
                hdc_conf_scores = trial_dict["eta"]
                for p in [.01, .1]:

                    try:
                        metrics = compute_metrics(y_pred=trial_dict["y_pred"], 
                                                  y_score=hdc_conf_scores, 
                                                  y_true=model_data_dict["y_test"],
                                                 p=p)

    #                     model_metric_dict["model"].append(model)
                        model_metric_dict["target"].append(target)
    #                     model_metric_dict["enrich-1"].append(metrics["enrich-1"])
    #                     model_metric_dict["enrich-10"].append(metrics["enrich-10"])
#                         import pdb
#                         pdb.set_trace()
                        model_metric_dict["test_time"].append(trial_dict["test_time"])
                        model_metric_dict["enrich"].append(metrics["enrich"])
                        model_metric_dict["p"].append(p)
                        model_metric_dict["seed"].append(seed)
                        model_metric_dict["precision"].append(metrics["precision"])
                        model_metric_dict["recall"].append(metrics["recall"])
                        model_metric_dict["f1"].append(metrics["f1"])
    #                     model_metric_dict["roc"].append(metrics["roc"])
                        model_metric_dict["model"].append(model_name)
                    except Exception as e:
                        print(e)

#                 #TODO: implement this 
#                 model_metric_dict["train_time"].append(train_time)

    return model_metric_dict
        


# VINA result

In [ ]:
from pathlib import Path
def docking_main(nrows=None):
    root_p = Path("/p/lustre2/ahashare/zhang30/LIT-PCBA-Data/")

    path_list = [path for path in root_p.glob("*-actives.csv")]


    df_list = []

    for idx, path in tqdm(enumerate(path_list), total=len(path_list)):
        print(idx, path)
        
#         '''
        target = path.name.split(".")[0].split("-")[0]
        print(target, idx+1, path)
        
        

        merged_df = None
        merged_df_path = Path(f"./lit_pcba_docking_analysis/{target}.csv")
        
        if not merged_df_path.exists():
            # can use the set of smiles in each result file
            
            # todo (10/18/23): is this valid?
            result_pkl = Path(f"/g/g13/jones289/workspace/hd-cuda-master/hdpy/hdpy/results/124/lit_pcba.{target}.ecfp.atomwise.0.pkl")        


            target_train_smiles_list = []
            target_test_smiles_list = []

            with open(result_pkl, "rb") as handle:

                data = pickle.load(handle)

                target_train_smiles_list = data["smiles_train"]
                target_test_smiles_list = data["smiles_test"]

                print(f"total of {len(target_train_smiles_list)} in training set, total of {len(target_test_smiles_list)} in testing set.")

            df_cols = ['file', ' scores/1', ' ligName']
            active_df = pd.read_csv(root_p / Path(f"{target}-actives.csv.clean"), sep=",", usecols=df_cols, nrows=nrows)
            active_df['y_true'] = [1] * len(active_df)



            inactive_df = pd.read_csv(root_p / Path(f"{target}-inactives.csv.clean"), sep=",", usecols=df_cols, nrows=nrows)
            inactive_df['y_true'] = [0] * len(inactive_df)

            target_df = pd.concat([active_df, inactive_df])
            # this will search over all of the docking results for each target, across each of the multiple protein models

            active_smiles_df = pd.read_csv(f"/p/vast1/jones289/lit_pcba/{target}/actives.smi", delim_whitespace=True, header=None)
            inactive_smiles_df = pd.read_csv(f"/p/vast1/jones289/lit_pcba/{target}/inactives.smi", delim_whitespace=True, header=None)
            target_smiles_df = pd.concat([active_smiles_df, inactive_smiles_df])



            top_pose_target_df = target_df.groupby([' ligName'], as_index=False)[[' ligName', ' scores/1', 'y_true']].min()


        
            merged_df_path.parent.mkdir(exist_ok=True, parents=True)
            merged_df = pd.merge(top_pose_target_df, target_smiles_df, left_on=" ligName", right_on=1)
            merged_df = merged_df[merged_df.apply(lambda x: x[0] in target_test_smiles_list, axis=1)]
            merged_df['target'] = [target] * len(merged_df)
            merged_df.to_csv(merged_df_path, index=False)
        else:
            merged_df = pd.read_csv(merged_df_path)
            
            if 'target' not in merged_df.columns:
                merged_df['target'] = [target] * len(merged_df)
                merged_df.to_csv(merged_df_path, index=False)



        df_list.append(merged_df)

    df = pd.concat(df_list)
    return df

In [ ]:
def compute_model_metric_df():
    model_metric_dict = aggregate_results(dataset="lit-pcba",multistep_p_list=[.05, .5],  
                                          multistep_sklearn_model="rf", 
                                         multistep_initial_p=.2)

    model_metric_df = pd.DataFrame({key: value for key, value in model_metric_dict.items() if key not in ["train_time", "test_time", "precision", "recall", "f1"]})

    # model_metric_df

    #######
    vina_result = docking_main(nrows=None)
    vina_result
    ###########

    docking_dict = {"enrich": [], "p": [], "model": [], "target": []}

    vina_enrich_list = []
    target_list = []
    vina_col=' scores/1'
    for target, target_df in vina_result.groupby("target"):

        for p in [.1, .01]:
            enrich = compute_enrichment_factor(scores=np.abs(target_df[vina_col]), 
                                      labels=target_df["y_true"], 
                                      n_percent=p)

            docking_dict["enrich"].append(float(enrich))
            docking_dict["p"].append(p)
            docking_dict["model"].append("Vina")
            docking_dict["target"].append(target)


    ##################
    model_metric_df = pd.concat([model_metric_df, pd.DataFrame(docking_dict)])
    
    # Backup the calculation
    model_metric_df.to_csv("fixed_litpcba_model_metric_df.csv")
    
    return model_metric_df    


In [ ]:

model_metric_df = compute_model_metric_df()


In [ ]:
model_metric_df["model"]

In [ ]:
(model_metric_df).groupby(["model", "target", "p"])["enrich"].describe().to_csv("summary_ef_stats.csv")

In [ ]:
def make_box_plot(enrich_1_df, enrich_10_df):
    
    enrich_f, enrich_ax = plt.subplots(2,1, figsize=(12,10), sharex=True, sharey=False)
    enrich_ax = enrich_ax.flatten()
    enrich_1_ax, enrich_10_ax = enrich_ax[0], enrich_ax[1]
    
#     sns.boxplot(data=enrich_1_df, x="model", y="enrich", ax=enrich_1_ax, palette=color_dict)
    sns.swarmplot(data=enrich_1_df, x="model", y="enrich", ax=enrich_1_ax)
    enrich_1_ax.set_title("(a) LIT-PCBA Enrichment at 1\%", fontdict={"fontsize": 18})
    enrich_1_ax.set_xlabel("")
    enrich_1_ax.set_ylabel("")
    enrich_1_ax.tick_params(axis="x", labelrotation=22.5)

    enrich_1_ax.set_ylabel("EF")

    plt.tight_layout()

    
#     sns.boxplot(data=enrich_10_df, x="model", y="enrich", ax=enrich_10_ax, palette=color_dict)
    sns.swarmplot(data=enrich_10_df, x="model", y="enrich", ax=enrich_10_ax)

    enrich_10_ax.set_title("(b) LIT-PCBA Enrichment at 10\%", fontdict={"fontsize": 18})
    enrich_10_ax.set_xlabel("")
    enrich_10_ax.set_ylabel("")
    enrich_10_ax.tick_params(axis="x", labelrotation=22.5)
    labels = [item.get_text() for item in enrich_10_ax.get_xticklabels()]
#     labels = [model_name_dict[x.get_text()] for x in enrich_10_ax.get_xticklabels()]
#     labels[-1] = combo_model_name
#     enrich_10_ax.set_xticklabels(labels)
    enrich_ax[0].set_ylabel("EF")
    enrich_ax[1].set_ylabel("EF")

    plt.tight_layout()
    # enrich_10_f.savefig("enrich_10.png", dpi=600, bbox_inches="tight")
    # enrich_10_f
    
    enrich_f.savefig("lit-pcba-enrich.png", dpi=600, bbox_inches="tight")

#     return enrich_f

In [ ]:
# take the average over the random seeds dimension for each combo of MODEL X TARGET X P
grp_df = (model_metric_df).groupby(["model", "target", "p"])["enrich"].mean().reset_index()


In [ ]:
grp_df[grp_df["p"] == .1]

In [ ]:
grp_df[grp_df["p"] == .01]

In [ ]:
make_box_plot(enrich_1_df=grp_df[grp_df["p"] == .01], 
          enrich_10_df=grp_df[grp_df["p"] == .1])

- should add the average EF (median?) to the labels on the horizontal axis
- scale the point size by standard deviation?
- standard scale the molformer embeddings?